### Video Presentation Transcriber

Extracts visually distinct frames from a video and uses a local LLM
(via Ollama) to transcribe/describe the content. Also extracts and
transcribes audio using Whisper.

In [15]:
# """
# Video Presentation Transcriber
#
# Extracts visually distinct frames from a video and uses a local LLM
# (via Ollama) to transcribe/describe the content. Also extracts and
# transcribes audio using Whisper.
#
# Requirements:
#     pip install opencv-python numpy requests pillow faster-whisper
#
# Usage:
#     from video_transcriber import VideoTranscriber
#
#     transcriber = VideoTranscriber()
#
#     # Define regions to ignore (e.g., presenter picture-in-picture windows)
#     # Format: (x, y, width, height) as fractions of frame size (0-1)
#     ignore_regions = [
#         (0.0, 0.7, 0.2, 0.3),   # Bottom-left corner
#         (0.8, 0.7, 0.2, 0.3),   # Bottom-right corner
#     ]
#
#     results = transcriber.process_video(
#         "presentation.mp4",
#         ignore_regions=ignore_regions,
#     )
# """

In [16]:
import cv2
import numpy as np
import requests
import base64
import json
import subprocess
import tempfile
from pathlib import Path
from dataclasses import dataclass, field
from typing import Iterator, Optional
import os

In [17]:
@dataclass
class AudioSegment:
    """A segment of transcribed audio."""
    start_seconds: float
    end_seconds: float
    text: str


@dataclass
class FrameResult:
    """Holds a frame and its transcription."""
    frame_number: int
    timestamp_seconds: float
    image: np.ndarray
    transcription: Optional[str] = None
    audio_segments: list[AudioSegment] = field(default_factory=list)


@dataclass
class TranscriptResult:
    """Complete transcript with visual and audio components."""
    frames: list[FrameResult]
    audio_segments: list[AudioSegment]

In [18]:
class VideoTranscriber:
    def __init__(
        self,
        ollama_url: str = "http://localhost:11434",
        vision_model: str = "llava",
        whisper_model: str = "base",
        similarity_threshold: float = 0.92,
        min_frame_interval: int = 15,
        image_quality: int = 100,  # NEW: configurable image quality
        use_png: bool = False,  # NEW: option to use PNG (lossless)
        timeout: int = 300,  # NEW: timeout in seconds for vision model API calls
    ):
        """
        Initialize the video transcriber.

        Args:
            ollama_url: Base URL for Ollama API
            vision_model: Vision model to use (llava, bakllava, llava-llama3, etc.)
            whisper_model: Whisper model size (tiny, base, small, medium, large-v3)
            similarity_threshold: Frames more similar than this are considered duplicates (0-1)
            min_frame_interval: Minimum frames between captures (avoids transition frames)
            image_quality: JPEG quality (1-100, higher is better, default 100)
            use_png: Use PNG encoding instead of JPEG (lossless but larger)
            timeout: Timeout in seconds for vision model API calls (default 300)
        """
        self.ollama_url = ollama_url.rstrip("/")
        self.vision_model = vision_model
        self.whisper_model = whisper_model
        self.similarity_threshold = similarity_threshold
        self.min_frame_interval = min_frame_interval
        self.image_quality = image_quality
        self.use_png = use_png
        self.timeout = timeout
        self._whisper = None

    def _get_whisper(self):
        """Lazy-load Whisper model."""
        if self._whisper is None:
            try:
                from faster_whisper import WhisperModel
                print(f"Loading Whisper model: {self.whisper_model}")
                # Use CPU by default; change to "cuda" if you have GPU
                self._whisper = WhisperModel(
                    self.whisper_model,
                    device="auto",
                    compute_type="auto",
                )
            except ImportError:
                raise ImportError(
                    "faster-whisper not installed. Run: pip install faster-whisper"
                )
        return self._whisper

    def _apply_ignore_mask(
        self,
        frame: np.ndarray,
        ignore_regions: list[tuple[float, float, float, float]],
    ) -> np.ndarray:
        """
        Apply mask to ignore specified regions (e.g., presenter windows).

        Args:
            frame: Input frame
            ignore_regions: List of (x, y, width, height) as fractions (0-1)

        Returns:
            Frame with ignored regions blacked out
        """
        if not ignore_regions:
            return frame

        masked = frame.copy()
        h, w = frame.shape[:2]

        for (rx, ry, rw, rh) in ignore_regions:
            x1 = int(rx * w)
            y1 = int(ry * h)
            x2 = int((rx + rw) * w)
            y2 = int((ry + rh) * h)

            # Black out the region
            masked[y1:y2, x1:x2] = 0

        return masked

    def _compute_frame_hash(self, frame: np.ndarray, hash_size: int = 16) -> np.ndarray:
        """
        Compute a perceptual hash for change detection.
        Uses average hash - fast and effective for slide detection.
        """
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        resized = cv2.resize(gray, (hash_size, hash_size), interpolation=cv2.INTER_AREA)
        mean_val = resized.mean()
        return (resized > mean_val).flatten()

    def _frames_similar(self, hash1: np.ndarray, hash2: np.ndarray) -> float:
        """Compute similarity between two frame hashes (0-1)."""
        return np.mean(hash1 == hash2)

    def preview_ignore_regions(
        self,
        video_path: str,
        ignore_regions: list[tuple[float, float, float, float]],
        output_path: str = "preview_mask.jpg",
        frame_number: int = 100,
    ):
        """
        Save a preview image showing which regions will be ignored.
        Useful for tuning the ignore_regions parameter.

        Args:
            video_path: Path to video
            ignore_regions: Regions to ignore
            output_path: Where to save preview image
            frame_number: Which frame to use for preview
        """
        cap = cv2.VideoCapture(video_path)
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
        ret, frame = cap.read()
        cap.release()

        if not ret:
            raise ValueError(f"Could not read frame {frame_number}")

        # Draw rectangles showing ignored regions
        h, w = frame.shape[:2]
        overlay = frame.copy()

        for (rx, ry, rw, rh) in ignore_regions:
            x1 = int(rx * w)
            y1 = int(ry * h)
            x2 = int((rx + rw) * w)
            y2 = int((ry + rh) * h)

            # Red semi-transparent overlay
            cv2.rectangle(overlay, (x1, y1), (x2, y2), (0, 0, 255), -1)

        # Blend overlay
        preview = cv2.addWeighted(overlay, 0.4, frame, 0.6, 0)

        # Add text labels
        cv2.putText(
            preview, "Red regions will be IGNORED for change detection",
            (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2
        )

        cv2.imwrite(output_path, preview)
        print(f"Preview saved to: {output_path}")

        return preview

    def extract_distinct_frames(
        self,
        video_path: str,
        sample_interval: int = 30,
        ignore_regions: Optional[list[tuple[float, float, float, float]]] = None,
    ) -> Iterator[FrameResult]:
        """
        Extract visually distinct frames from video.

        Args:
            video_path: Path to video file
            sample_interval: Check every N frames for changes
            ignore_regions: Regions to mask out before change detection

        Yields:
            FrameResult objects for each distinct frame
        """
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            raise ValueError(f"Could not open video: {video_path}")

        fps = cap.get(cv2.CAP_PROP_FPS)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        duration = total_frames / fps

        print(f"Video: {total_frames} frames at {fps:.1f} FPS ({duration:.1f}s / {duration/60:.1f}min)")
        if ignore_regions:
            print(f"Ignoring {len(ignore_regions)} region(s) for change detection")

        last_hash = None
        last_captured_frame = -self.min_frame_interval
        frame_count = 0

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            frame_count += 1

            # Only check at sample intervals
            if frame_count % sample_interval != 0:
                continue

            # Apply mask before computing hash (but keep original frame for output)
            masked_frame = self._apply_ignore_mask(frame, ignore_regions or [])
            current_hash = self._compute_frame_hash(masked_frame)

            # Check if frame is sufficiently different
            is_distinct = False
            if last_hash is None:
                is_distinct = True
            else:
                similarity = self._frames_similar(current_hash, last_hash)
                if similarity < self.similarity_threshold:
                    if (frame_count - last_captured_frame) >= self.min_frame_interval:
                        is_distinct = True

            if is_distinct:
                timestamp = frame_count / fps
                print(f"  Captured frame {frame_count} at {timestamp:.1f}s ({timestamp/60:.1f}min)")

                yield FrameResult(
                    frame_number=frame_count,
                    timestamp_seconds=timestamp,
                    image=frame.copy(),  # Original frame, not masked
                )

                last_hash = current_hash
                last_captured_frame = frame_count

        cap.release()

    def extract_audio(self, video_path: str, output_path: Optional[str] = None) -> str:
        """
        Extract audio from video using ffmpeg.

        Args:
            video_path: Path to video file
            output_path: Where to save audio (default: temp file)

        Returns:
            Path to extracted audio file
        """
        if output_path is None:
            output_path = tempfile.mktemp(suffix=".wav")

        print(f"Extracting audio from video...")

        cmd = [
            "ffmpeg", "-y",
            "-i", video_path,
            "-vn",  # No video
            "-acodec", "pcm_s16le",  # PCM format for Whisper
            "-ar", "16000",  # 16kHz sample rate
            "-ac", "1",  # Mono
            output_path
        ]

        result = subprocess.run(
            cmd,
            capture_output=True,
            text=True,
        )

        if result.returncode != 0:
            raise RuntimeError(f"ffmpeg failed: {result.stderr}")

        print(f"Audio extracted to: {output_path}")
        return output_path

    def transcribe_audio(self, audio_path: str) -> list[AudioSegment]:
        """
        Transcribe audio using Whisper.

        Args:
            audio_path: Path to audio file

        Returns:
            List of AudioSegment with timestamps and text
        """
        print(f"Transcribing audio with Whisper ({self.whisper_model})...")

        model = self._get_whisper()
        segments, info = model.transcribe(
            audio_path,
            beam_size=5,
            word_timestamps=False,
        )

        print(f"Detected language: {info.language} (probability: {info.language_probability:.2f})")

        results = []
        for segment in segments:
            results.append(AudioSegment(
                start_seconds=segment.start,
                end_seconds=segment.end,
                text=segment.text.strip(),
            ))

        print(f"Transcribed {len(results)} audio segments")
        return results

    def _encode_image(self, image: np.ndarray) -> str:
        """Encode image to base64 for Ollama API with high quality."""
        if self.use_png:
            # PNG: lossless but larger file size
            _, buffer = cv2.imencode('.png', image)
        else:
            # JPEG: configurable quality (default 100 for maximum quality)
            _, buffer = cv2.imencode('.jpg', image, [cv2.IMWRITE_JPEG_QUALITY, self.image_quality])
        return base64.b64encode(buffer).decode('utf-8')

    def transcribe_frame(
        self,
        image: np.ndarray,
        prompt: str = "Transcribe all text visible in this presentation slide. Include headings, bullet points, and any other text. Format it clearly.",
    ) -> str:
        """
        Use local LLM to transcribe/describe a frame.
        """
        encoded = self._encode_image(image)

        payload = {
            "model": self.vision_model,
            "prompt": prompt,
            "images": [encoded],
            "stream": False,
        }

        response = requests.post(
            f"{self.ollama_url}/api/generate",
            json=payload,
            timeout=self.timeout,
        )
        response.raise_for_status()

        return response.json().get("response", "")

    def _merge_audio_with_frames(
        self,
        frames: list[FrameResult],
        audio_segments: list[AudioSegment],
    ) -> list[FrameResult]:
        """
        Associate audio segments with their corresponding frames based on timestamps.
        Each frame gets the audio segments that occur between it and the next frame.
        """
        if not audio_segments:
            return frames

        for i, frame in enumerate(frames):
            # Find time range for this frame
            start_time = frame.timestamp_seconds
            if i + 1 < len(frames):
                end_time = frames[i + 1].timestamp_seconds
            else:
                # Last frame: include all remaining audio
                end_time = float('inf')

            # Find audio segments in this time range
            frame.audio_segments = [
                seg for seg in audio_segments
                if seg.start_seconds >= start_time and seg.start_seconds < end_time
            ]

        return frames

    def process_video(
        self,
        video_path: str,
        sample_interval: int = 30,
        ignore_regions: Optional[list[tuple[float, float, float, float]]] = None,
        prompt: Optional[str] = None,
        output_dir: Optional[str] = None,
        transcribe_audio: bool = True,
        transcribe_visuals: bool = True,
    ) -> TranscriptResult:
        """
        Process entire video: extract distinct frames, transcribe visuals and audio.

        Args:
            video_path: Path to video file
            sample_interval: Check every N frames for changes
            ignore_regions: Regions to mask for change detection (presenter windows)
            prompt: Custom prompt for visual transcription
            output_dir: If provided, save frames as images here
            transcribe_audio: Whether to transcribe audio with Whisper
            transcribe_visuals: Whether to transcribe visuals with vision LLM

        Returns:
            TranscriptResult with frames and audio segments
        """
        if output_dir:
            output_path = Path(output_dir)
            output_path.mkdir(parents=True, exist_ok=True)

        # Extract audio first (if requested)
        audio_segments = []
        if transcribe_audio:
            try:
                audio_path = self.extract_audio(video_path)
                audio_segments = self.transcribe_audio(audio_path)
                # Clean up temp audio file
                if audio_path.startswith(tempfile.gettempdir()):
                    os.remove(audio_path)
            except Exception as e:
                print(f"Warning: Audio transcription failed: {e}")
                print("Continuing with visual transcription only...")

        # Extract and transcribe frames
        frames = []
        for frame_result in self.extract_distinct_frames(
            video_path, sample_interval, ignore_regions
        ):
            if transcribe_visuals:
                print(f"  Transcribing frame {frame_result.frame_number} visuals...")
                if prompt:
                    frame_result.transcription = self.transcribe_frame(frame_result.image, prompt)
                else:
                    frame_result.transcription = self.transcribe_frame(frame_result.image)

            if output_dir:
                img_path = output_path / f"frame_{frame_result.frame_number:06d}.jpg"
                cv2.imwrite(str(img_path), frame_result.image)

            frames.append(frame_result)

        # Merge audio with frames
        frames = self._merge_audio_with_frames(frames, audio_segments)

        return TranscriptResult(frames=frames, audio_segments=audio_segments)

    def save_transcript(
        self,
        result: TranscriptResult,
        output_path: str,
        include_visuals: bool = True,
        include_audio: bool = True,
    ):
        """Save transcription results to a text file."""
        with open(output_path, 'w') as f:
            f.write("VIDEO PRESENTATION TRANSCRIPT\n")
            f.write("=" * 60 + "\n\n")

            for frame in result.frames:
                minutes = int(frame.timestamp_seconds // 60)
                seconds = int(frame.timestamp_seconds % 60)

                f.write(f"\n{'='*60}\n")
                f.write(f"[{minutes:02d}:{seconds:02d}] SLIDE (Frame {frame.frame_number})\n")
                f.write("=" * 60 + "\n")

                if include_visuals and frame.transcription:
                    f.write("\n📊 SLIDE CONTENT:\n")
                    f.write("-" * 40 + "\n")
                    f.write(frame.transcription)
                    f.write("\n")

                if include_audio and frame.audio_segments:
                    f.write("\n🎤 SPEAKER AUDIO:\n")
                    f.write("-" * 40 + "\n")
                    for seg in frame.audio_segments:
                        seg_min = int(seg.start_seconds // 60)
                        seg_sec = int(seg.start_seconds % 60)
                        f.write(f"[{seg_min:02d}:{seg_sec:02d}] {seg.text}\n")

                f.write("\n")

        print(f"Transcript saved to: {output_path}")

    def save_audio_only_transcript(
        self,
        result: TranscriptResult,
        output_path: str,
    ):
        """Save just the audio transcript (like a subtitle file)."""
        with open(output_path, 'w') as f:
            f.write("AUDIO TRANSCRIPT\n")
            f.write("=" * 60 + "\n\n")

            for seg in result.audio_segments:
                start_min = int(seg.start_seconds // 60)
                start_sec = int(seg.start_seconds % 60)
                end_min = int(seg.end_seconds // 60)
                end_sec = int(seg.end_seconds % 60)

                f.write(f"[{start_min:02d}:{start_sec:02d} -> {end_min:02d}:{end_sec:02d}]\n")
                f.write(f"{seg.text}\n\n")

        print(f"Audio transcript saved to: {output_path}")

In [19]:


# # Example usage
# if __name__ == "__main__":
#     import sys
#
#     if len(sys.argv) < 2:
#         print("Usage: python video_transcriber.py <video_file> [output_dir]")
#         print("\nExample:")
#         print("  python video_transcriber.py presentation.mp4 ./output")
#         print("\nTo preview ignore regions:")
#         print("  python video_transcriber.py presentation.mp4 --preview")
#         sys.exit(1)
#
#     video_file = sys.argv[1]
#
#     # Example ignore regions for presenter windows
#     # Adjust these coordinates based on your video layout!
#     # Format: (x, y, width, height) as fractions of frame size (0-1)
#     ignore_regions = [
#         # Example: Bottom-right corner presenter window
#         # (0.75, 0.70, 0.25, 0.30),
#         # Example: Bottom-left corner presenter window
#         # (0.0, 0.70, 0.25, 0.30),
#     ]
#
#     transcriber = VideoTranscriber(
#         vision_model="llava",
#         whisper_model="base",  # Options: tiny, base, small, medium, large-v3
#         similarity_threshold=0.90,
#     )
#
#     # Preview mode
#     if "--preview" in sys.argv:
#         # First, show frame without regions to help identify coordinates
#         transcriber.preview_ignore_regions(
#             video_file,
#             ignore_regions=[],  # Empty to see full frame
#             output_path="preview_full.jpg",
#         )
#         if ignore_regions:
#             transcriber.preview_ignore_regions(
#                 video_file,
#                 ignore_regions,
#                 output_path="preview_masked.jpg",
#             )
#         print("\nCheck the preview images to tune your ignore_regions.")
#         sys.exit(0)
#
#     output_dir = sys.argv[2] if len(sys.argv) > 2 else "./video_output"
#
#     print(f"Processing: {video_file}")
#     result = transcriber.process_video(
#         video_file,
#         sample_interval=30,
#         ignore_regions=ignore_regions,
#         output_dir=output_dir,
#     )
#
#     # Save combined transcript
#     transcriber.save_transcript(result, f"{output_dir}/transcript.txt")
#
#     # Save audio-only transcript
#     transcriber.save_audio_only_transcript(result, f"{output_dir}/audio_transcript.txt")
#
#     print(f"\nDone! Captured {len(result.frames)} distinct slides.")
#    print(f"Transcribed {len(result.audio_segments)} audio segments.")

IndentationError: unexpected indent (3015724656.py, line 65)

In [21]:
# Test with cp-demo.mp4 (no audio)
video_file = "../data/cp-demo.mp4"
output_dir = "../tests/output/cp-demo"

transcriber = VideoTranscriber(
    ollama_url="http://polwarth:11434",
    vision_model="deepseek-ocr:latest",
    whisper_model="base",
    similarity_threshold=0.90,
    image_quality=100,  # Maximum JPEG quality
    use_png=False,  # Set to True for lossless (but larger files)
    timeout=600,  # 10 minutes timeout for slow models
)

# Better prompt for OCR-focused models
ocr_prompt = """Extract and transcribe ALL text visible in this image. 
Be precise and accurate. Do not add any interpretation or description.
Only output the text you can see, preserving the layout and structure."""

print(f"Processing: {video_file}")
result = transcriber.process_video(
    video_file,
    sample_interval=3,
    ignore_regions=[],  # No presenter window to ignore
    output_dir=output_dir,
    transcribe_audio=False,  # No audio in this video
    transcribe_visuals=True,
    prompt=ocr_prompt,
)

# Save transcript
transcriber.save_transcript(result, f"{output_dir}/transcript.txt")

print(f"\nDone! Captured {len(result.frames)} distinct slides.")
print(f"Output saved to: {output_dir}")

Processing: ../data/cp-demo.mp4
Video: 2186 frames at 15.0 FPS (145.7s / 2.4min)
  Captured frame 3 at 0.2s (0.0min)
  Transcribing frame 3 visuals...
  Captured frame 78 at 5.2s (0.1min)
  Transcribing frame 78 visuals...
  Captured frame 438 at 29.2s (0.5min)
  Transcribing frame 438 visuals...
  Captured frame 612 at 40.8s (0.7min)
  Transcribing frame 612 visuals...
  Captured frame 651 at 43.4s (0.7min)
  Transcribing frame 651 visuals...
  Captured frame 1359 at 90.6s (1.5min)
  Transcribing frame 1359 visuals...
  Captured frame 1374 at 91.6s (1.5min)
  Transcribing frame 1374 visuals...
  Captured frame 1551 at 103.4s (1.7min)
  Transcribing frame 1551 visuals...
  Captured frame 1707 at 113.8s (1.9min)
  Transcribing frame 1707 visuals...
Transcript saved to: ../tests/output/cp-demo/transcript.txt

Done! Captured 9 distinct slides.
Output saved to: ../tests/output/cp-demo


In [14]:
# Check video properties (resolution, etc.)
import cv2
cap = cv2.VideoCapture("../data/cp-demo.mp4")
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
cap.release()
print(f"Video resolution: {width}x{height}")
print(f"FPS: {fps}")
print(f"Aspect ratio: {width/height:.2f}")
if width < 1280:
    print("⚠️ Warning: Low resolution video may affect OCR quality")

Video resolution: 1280x972
FPS: 15.0
Aspect ratio: 1.32
